In [1]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
import numpy as np
model = SentenceTransformer('all-MiniLM-L6-v2')

/home/poloniki/.pyenv/versions/3.8.12/envs/quint/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('../quint/data/results/Short.txt') as f:
    doc = f.readlines()
    
doc = doc[0].replace("?", ". ")
sentences = doc.split('. ')

In [3]:
embeddings = model.encode(sentences)

In [52]:

# A =  embeddings
# A_sparse = sparse.csr_matrix(A)

# similarities = cosine_similarity(A_sparse)
# # print('pairwise dense output:\n {}\n'.format(similarities))

# #also can output sparse matrices
# similarities_sparse = cosine_similarity(A_sparse,dense_output=False)
# # print('pairwise sparse output:\n {}\n'.format(similarities_sparse))

In [14]:
# A =  embeddings[0:0+50]
# A_sparse = sparse.csr_matrix(A)
# similarities = cosine_similarity(A_sparse)
# average_sim = similarities.mean(axis=1)
# best_sentence = list(average_sim).index(average_sim.max())
# similarities_sparse = cosine_similarity(A_sparse,dense_output=False)
# print('pairwise sparse output:\n {}\n'.format(similarities_sparse))

In [53]:
# average_sim = np.round(similarities.mean(axis=1),2)
# np.round(average_sim,2)

In [4]:
import pandas as pd

df = pd.DataFrame()
df['sentence'] = sentences[:-1]

df['len'] = df['sentence'].apply(lambda x: len(x))
df['cum'] = df['len'].cumsum()

In [16]:
steps = round(df['cum'].sum() / 10)

In [17]:
best_sentences = []
steps = range(0, df.cum.max() + steps, steps*2)
for each in range(len(steps)-1):
    temp_df = df.loc[(df.cum > steps[each])&(df.cum < steps[each+1])]
    indexes = temp_df.index
    A =  embeddings[indexes[0]:indexes[-1]+1]
    A_sparse = sparse.csr_matrix(A)
    similarities = cosine_similarity(A_sparse)
    
    average_sim = list(np.round(similarities.mean(axis=1),2))
    min_score =np.min(average_sim)
    max_score = np.max(average_sim)
    best_sentence = [i for i, x in enumerate(average_sim) if (x > max_score-0.02)]
    for every in best_sentence:
        best_sentences.append(temp_df.iloc[every].sentence)
        
best_sentences = [each for each in best_sentences if len(each.split(' '))> 5]
df['highlight'] = df['sentence'].apply(lambda x: x in best_sentences)

In [18]:
best_sentences

[" They're taking money from the Russians",
 'And of course the answer is no but I do this for a living like I speak',
 'Do you want to do that']

In [19]:
df.loc[df.highlight==True]

,sentence,len,cum,highlight
1,They're taking money from the Russians,39,100,True
2,And of course the answer is no but I do this f...,70,170,True
10,Do you want to do that,22,591,True


In [21]:
import spacy
nlp = spacy.load("en_core_web_sm")

nlp.pipe('Here am i Nikita')
# mentioned = [ent.lemma_ for ent in doc.ents if ent.label_ == 'PERSON']

2022-09-05 11:39:36.185015: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-09-05 11:39:36.241424: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-05 11:39:36.241438: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-05 11:39:38.160082: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-05 11:39:38.160120: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to c

<generator object Language.pipe at 0x7fe4f2d02ba0>

In [23]:
import string

def preprocessing(sentence):

    sentence = ''.join([each for each in sentence if each not in string.punctuation])

    return sentence
def get_persons(text):
    for doc in nlp.pipe([text]):
        mentioned = [ent.lemma_ for ent in doc.ents if ent.label_ in ['PERSON', 'PRODUCT', 'ORG', 'GPE', 'MONEY', 'CARDINAL']]
    return mentioned
    
df['names'] = df.sentence.apply(get_persons)

from termcolor import colored
from spacy import displacy

text = ''
for num, each in enumerate(df['sentence']):
    #Bold words
    to_bold = df['names'].iloc[num]
    if len(to_bold)>0:
        to_unpack = [i.split(' ', 1) for i in to_bold]
        flat_list = [item for sublist in to_unpack for item in sublist]
        each = " ".join(f'\033[1m{t}\033[0m' if (preprocessing(t) in flat_list) & (preprocessing(t).lower() != 'the')  else t for t in each.split())
        #print(flat_list, each, each.split())
    # Highlight best sentence if it is there 
    if df['highlight'].iloc[num] == True:
        text+=colored(f'{each}. ','white','on_red')
    else:
        text+=f'{each}. '
    
print(text)

You know people are like how do you live and things like that.  They're taking money from the Russians. And of course the answer is no but I do this for a living like I speak. I don't have a YouTube channel that where it's you know, I'm Joe Rogan, but I give speeches at universities and things like that. I do a lot of interviews and so we're recording now right on set up cool. Is it possible that you could do a YouTube channel.  Would that work.  If yeah, like I I mean if you introduce me so like I get followers. Yeah, we could do that. I'm all in that that could absolutely happen. Do you want to do that.  Is it something you want to do. 


In [192]:
# from spacy import displacy

# doc = nlp(text)
# options = {"ents": ['PERSON', 'PRODUCT', 'ORG', 'GPE', 'MONEY', 'CARDINAL ']}

# displacy.serve(doc, style="ent",options=options)

In [75]:
similarities_sparse = cosine_similarity(A_sparse,dense_output=False)
print('pairwise sparse output:\n {}\n'.format(similarities_sparse))

pairwise sparse output:
   (0, 104)	-0.02589854
  (0, 103)	0.021266721
  (0, 102)	0.0074429987
  (0, 101)	0.005696579
  (0, 100)	0.0046391385
  (0, 99)	0.08170297
  (0, 98)	0.047602687
  (0, 97)	0.08170297
  (0, 96)	-0.016100569
  (0, 95)	0.04104335
  (0, 94)	0.052055288
  (0, 93)	0.040341523
  (0, 92)	0.28603432
  (0, 91)	0.12118529
  (0, 90)	0.1056238
  (0, 89)	0.12997746
  (0, 88)	0.09192037
  (0, 87)	0.0020450838
  (0, 86)	0.052878164
  (0, 85)	-0.0083633885
  (0, 84)	0.035215165
  (0, 83)	-0.001178159
  (0, 82)	0.105132595
  (0, 81)	0.17433318
  (0, 80)	0.21162403
  :	:
  (104, 24)	0.03408325
  (104, 23)	-0.07958412
  (104, 22)	-0.02097783
  (104, 21)	0.06452103
  (104, 20)	0.16395216
  (104, 19)	0.16861263
  (104, 18)	0.058462113
  (104, 17)	0.12590262
  (104, 16)	0.105187334
  (104, 15)	0.16208836
  (104, 14)	0.06427233
  (104, 13)	-0.074408196
  (104, 12)	-0.0098676905
  (104, 11)	0.07735178
  (104, 10)	0.037142176
  (104, 9)	-0.0681082
  (104, 8)	-0.01745296
  (104, 7)	0.02898